In [19]:
import numpy as np
from pyvo.dal import tap
from matplotlib import pyplot as plt

In [39]:
source = 'G359'
# source = 'G10'  
# source = 'G0'
# source = 'G1'

long = np.int32(source.split('G')[1])
source_left = f'G{long-1}'
source_right = f'G{long+1}'

long = int(source.split('G')[1])  # Convert to an integer
source_left = f'G{long-1:03d}'    # Format as 3-digit number with leading zeros
source_right = f'G{long+1:03d}'   # Format as 3-digit number with leading zeros
source_ = f'G{long:03d}'          # Format as 3-digit number with leading zeros
long_left = long - 1
long_right = long + 1

if long == 0:
    # Assuming long_left was computed as -1, adjust it to wrap around
    long_left = 360 + long_left  # For example, -1 becomes 359
    # long_right is assumed to be 1

    print(long_left, long_right)
    query = (
        "SELECT dp_id, exposure, prog_id, object, dp_tech, instrument, ra, dec, gal_lon, gal_lat \n"
        "FROM dbo.raw \n"
        "WHERE dp_id LIKE 'APEXHET.%' \n"
        "AND (prog_id LIKE '092.F-9315%' OR prog_id LIKE '193.C-0584%') \n"
        "AND (object LIKE 'G%') \n"
        "AND ((gal_lon >= {ll} AND gal_lon <= 360) OR (gal_lon >= 0 AND gal_lon <= {lr})) \n"
        "AND dp_cat = 'SCIENCE'"
    ).format(ll=long_left, lr=long_right)

else: 
    query = (\
        "SELECT dp_id, exposure, prog_id, object, dp_tech, instrument, ra, dec, gal_lon, gal_lat \n"
        "FROM dbo.raw \n"
        "WHERE dp_id LIKE 'APEXHET.%' \n"
        "AND (prog_id LIKE '092.F-9315%' OR prog_id LIKE '193.C-0584%') \n"
        # f"AND (object LIKE '{source_}%' OR object LIKE '{source_left}%' OR object LIKE '{source_right}%') \n"
        f"AND (object LIKE 'G%') \n"
        f"AND (gal_lon >= {long_left} AND gal_lon <= {long_right}) \n"
        "AND dp_cat = 'SCIENCE'"
    )

# print(query)
ESO_TAP_OBS = "http://archive.eso.org/tap_obs"
tapobs = tap.TAPService(ESO_TAP_OBS)
table = tapobs.search(query=query, maxrec=1000).to_table()
table.sort('gal_lon')
table.pprint_all()
# print(table['gal_lon'].value)

             dp_id              exposure    prog_id       object    dp_tech  instrument      ra          dec       gal_lon    gal_lat 
                                   s                                                        deg          deg         deg        deg   
------------------------------- -------- ------------- ------------ -------- ---------- ------------ ------------ ---------- ---------
APEXHET.2014-06-17T07:35:16.000 1046.373 193.C-0584(A) G358.12-0.25 SPECTRUM    APEXHET     265.5175 -30.66388805 358.125018  -0.24998
APEXHET.2017-09-04T21:22:50.000 1046.373 193.C-0584(A) G358.12+0.75 SPECTRUM    APEXHET 264.53541611 -30.13277694 358.125085  0.749792
APEXHET.2017-09-04T03:12:50.000 1046.373 193.C-0584(A) G358.12-0.75 SPECTRUM    APEXHET 266.01291611 -30.92666611 358.125238 -0.750164
APEXHET.2014-06-17T08:27:21.000 1046.373 193.C-0584(A) G358.25-0.38 SPECTRUM    APEXHET     265.7175 -30.62361111 358.249898 -0.375024
APEXHET.2013-11-14T21:42:49.000 2093.657 092.F-9315(A) 